In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from keras.metrics import Precision, Recall, F1Score

In [2]:
# connect, give access, and "download" your drive on the google VM
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# change to your path to the dataset in drive (create a shortcut of shared ML_project in "MyDrive")
train_path = '/content/drive/MyDrive/ML_project/RNN/train_set_not_std_less_noise.parquet'
test_path = '/content/drive/MyDrive/ML_project/test_set_not_std.parquet'

In [4]:
train_set = pd.read_parquet(train_path, engine='pyarrow').reset_index(drop=True)
test_set = pd.read_parquet(test_path, engine='pyarrow').reset_index(drop=True)

We want to truncated videos, to have the same length

In [5]:
# Find shortest video in both sets
train_video_lengths = train_set.groupby('video_id').size().min()
test_video_lengths = test_set.groupby('video_id').size().min()

# Take the minimum of both sets
min_video_length = min(train_video_lengths, test_video_lengths)

# Truncate train_set
train_set = train_set.groupby('video_id').apply(lambda group: group.head(min_video_length)).reset_index(drop=True)

# Truncate test_set
test_set = test_set.groupby('video_id').apply(lambda group: group.head(min_video_length)).reset_index(drop=True)

In [6]:
# Create label dataframes for train and test set
y_train = train_set[['video_id','Exercise','Set']].copy().drop_duplicates()
y_test = test_set[['video_id','Exercise','Set']].copy().drop_duplicates()

# One-hot encode labels
y_train = pd.get_dummies(y_train, columns=['Set'], prefix='Onehot')
y_test = pd.get_dummies(y_test, columns=['Set'], prefix='Onehot')

# Drop useless columns and convert to numpy array
y_train = y_train.drop(columns=['video_id', 'Exercise']).to_numpy(dtype=np.float32)
y_test = y_test.drop(columns=['video_id', 'Exercise']).to_numpy(dtype=np.float32)

In [7]:
# Get columns used for prediction
feature_columns = train_set.columns.to_list()[5:]

# Convert train_set and test_set to NumPy arrays
X_train = np.array([group[feature_columns].astype(np.float32) for _, group in train_set.groupby('video_id')])
X_test = np.array([group[feature_columns].astype(np.float32) for _, group in test_set.groupby('video_id')])

In [8]:
del train_set
del test_set

In [9]:
print(X_train.shape, y_train.shape)

(3230, 345, 107) (3230, 7)


In [10]:
# Modeling constants:
M_TRAIN = X_train.shape[0]
M_TEST = X_test.shape[0]
N = X_train.shape[2]
T = X_train.shape[1]
BATCH = 150
LR = 5e-4

# Build the Model
model = Sequential()
model.add(
    LSTM(
        input_shape=(T, N), units=128,
               activation='tanh', recurrent_activation='sigmoid'))

model.add(Dense(128, activation='relu'))

model.add(Dense(7, activation='softmax'))

# Compile the model with Adam optimizer
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy',F1Score()],
              optimizer=Adam(learning_rate=LR))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               120832    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 138247 (540.03 KB)
Trainable params: 138247 (540.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=250,
    batch_size=64,
    validation_split=0.0,
    validation_data=(X_test[:M_TEST], y_test[:M_TEST]),
    shuffle=False
)

Epoch 1/50
51/51 [==============================] - 2s 32ms/step - loss: 0.2133 - accuracy: 0.9350 - f1_score: 0.9397 - val_loss: 3.1698 - val_accuracy: 0.4286 - val_f1_score: 0.3470
Epoch 2/50
51/51 [==============================] - 1s 21ms/step - loss: 0.1254 - accuracy: 0.9610 - f1_score: 0.9651 - val_loss: 3.2018 - val_accuracy: 0.4262 - val_f1_score: 0.3440
Epoch 3/50
51/51 [==============================] - 1s 21ms/step - loss: 0.0927 - accuracy: 0.9737 - f1_score: 0.9755 - val_loss: 3.4051 - val_accuracy: 0.4214 - val_f1_score: 0.3393
Epoch 4/50
51/51 [==============================] - 1s 22ms/step - loss: 0.0748 - accuracy: 0.9771 - f1_score: 0.9802 - val_loss: 3.3969 - val_accuracy: 0.4286 - val_f1_score: 0.3453
Epoch 5/50
51/51 [==============================] - 1s 22ms/step - loss: 0.1144 - accuracy: 0.9724 - f1_score: 0.9742 - val_loss: 3.4743 - val_accuracy: 0.4000 - val_f1_score: 0.3387
Epoch 6/50
51/51 [==============================] - 1s 22ms/step - loss: 0.1289 - acc

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

14/14 [==============================] - 0s 28ms/step - loss: 3.8368 - accuracy: 0.4810
Test Accuracy: 48.10%
